<a href="https://colab.research.google.com/github/PaoloMadda01/Projects-with-Google-Colab/blob/main/CODEF_TF_Modello.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Utilizzo un modello personale (GITHUB)

In [ ]:
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 10.3 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG19
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_hub as hub
import cv2
import numpy as np
import imageio
import time
from PIL import Image
from google.colab import files
import tensorflow as tf
from google.colab import files
import cv2
import numpy as np

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:


# Carica l'immagine dal tuo computer
uploaded = files.upload()
style_image_path = list(uploaded.keys())[0]

# Apri l'immagine usando PIL
style_image_pil = Image.open(style_image_path)

# Converti l'immagine PIL in un array NumPy
style_image_np = np.array(style_image_pil)

# Normalizza l'immagine se necessario (ad esempio, scala i valori dei pixel a [0, 1])
# Questo passaggio potrebbe non essere necessario a seconda del tuo modello
style_image_np = style_image_np / 255.0

# Converti l'array NumPy in un tensore TensorFlow
style_image_tensor = tf.convert_to_tensor(style_image_np, dtype=tf.float32)

# Aggiungi una dimensione batch all'inizio se necessario
# Questo dipenderà dal tuo modello; alcuni modelli richiedono una dimensione batch
style_image_tensor = tf.expand_dims(style_image_tensor, axis=0)






Saving anime.jpeg to anime (2).jpeg


In [ ]:


class CustomStyleTransfer(Model):
    def __init__(self, style_weight=1e-2, content_weight=1e-4):
        super(CustomStyleTransfer, self).__init__()
        self.style_weight = style_weight
        self.content_weight = content_weight

        # Utilizziamo la rete VGG19 pre-addestrata come estrattore di caratteristiche
        vgg = VGG19(include_top=False, weights='imagenet')
        vgg.trainable = False

        # Selezioniamo gli strati specifici per il contenuto e lo stile
        self.content_layers = ['block4_conv2']
        self.style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']

        self.num_content_layers = len(self.content_layers)
        self.num_style_layers = len(self.style_layers)

        # Creiamo un modello che restituisce gli output di strati specifici
        outputs = [vgg.get_layer(name).output for name in (self.content_layers + self.style_layers)]
        self.vgg = Model(inputs=vgg.input, outputs=outputs)

    def call(self, inputs):
        content_image, style_image = inputs

        # Preprocessa le immagini per VGG19
        content_image = tf.keras.applications.vgg19.preprocess_input(content_image * 255)
        style_image = tf.keras.applications.vgg19.preprocess_input(style_image * 255)

        # Estrae le caratteristiche del contenuto e dello stile
        content_outputs = self.vgg(content_image, training=False)
        style_outputs = self.vgg(style_image, training=False)

        content_dict = {name: value for name, value in zip(self.content_layers, content_outputs[:self.num_content_layers])}
        style_dict = {name: value for name, value in zip(self.style_layers, style_outputs[self.num_content_layers:])}

        # Calcola la perdita di contenuto
        content_loss = tf.add_n([tf.reduce_mean((content_dict[name] - style_dict.get(name, tf.zeros_like(content_dict[name]))) ** 2)
                         for name in self.content_layers])


        content_loss *= self.content_weight / self.num_content_layers

        # Calcola la perdita di stile
        style_loss = tf.add_n([self.gram_matrix_loss(style_dict[name], style_dict[name])
                       for name in self.style_layers])

        style_loss *= self.style_weight / self.num_style_layers

        # Combina le perdite di contenuto e stile
        total_loss = content_loss + style_loss

# Utilizza un ottimizzatore per minimizzare la perdita e generare l'immagine stilizzata
        optimizer = tf.optimizers.Adam(learning_rate=0.02)

# Definisci la variabile per l'immagine stilizzata, inizializzata con l'immagine di contenuto
        stylized_image = tf.Variable(content_image)
        return stylized_image



    def get_content_features(self, image):
        vgg_outputs = self.vgg(image, training=False)
        content_dict = {layer.name: output for layer, output in zip(self.vgg.layers, vgg_outputs) if layer.name in self.content_layers}
        return content_dict

    def get_style_features(self, image):
        vgg_outputs = self.vgg(image, training=False)
        style_dict = {layer.name: output for layer, output in zip(self.vgg.layers, vgg_outputs) if layer.name in self.style_layers}
        return style_dict




# Definisci una funzione di allenamento che esegue una singola iterazione di ottimizzazione
    def train_step(self):
      with tf.GradientTape() as tape:
        # Calcola le perdite di contenuto e stile per l'immagine stilizzata corrente
        style_output, content_output = self(stylized_image)
        style_loss = self.compute_style_loss(style_output)
        content_loss = self.compute_content_loss(content_output)
        style_loss *= self.style_weight / self.num_style_layers
        total_loss = content_loss + style_loss

    # Calcola i gradienti
      grads = tape.gradient(total_loss, stylized_image)

    # Applica i gradienti per aggiornare l'immagine stilizzata
      self.optimizer.apply_gradients([(grads, stylized_image)])


    def train_model(self, num_iterations=10):  # Definisci un valore predefinito per num_iterations, se desiderato
        for i in range(num_iterations):
            self.train_step()


    def gram_matrix_loss(self, style_output, content_output):
    # Calcola la matrice di Gram per la perdita di stile
      S_gram = self.gram_matrix(style_output)
      C_gram = self.gram_matrix(content_output)
      channels = tf.cast(style_output.shape[-1], tf.float32)  # Cast to float32
      size = tf.cast(tf.size(style_output), tf.float32)  # Cast to float32
      loss = tf.reduce_sum(tf.square(S_gram - C_gram)) / (4.0 * (channels ** 2) * (size ** 2))
      return loss


    def gram_matrix(self, input_tensor):
    # La funzione gram_matrix calcola la matrice di Gram di un tensore di input
    # Il tensore di input è ridimensionato a 2D (forma: channels, width*height) e trasposto
      channels = input_tensor.shape[-1]
      a = tf.reshape(input_tensor, [-1, channels])
      n = tf.shape(a)[0]
      gram = tf.matmul(a, a, transpose_a=True)
      return gram / tf.cast(n, tf.float32)


# Crea un'istanza del modello di trasferimento di stile personalizzato
model = CustomStyleTransfer(style_weight=1e-2, content_weight=1e-4)
model.train_model(num_iterations=500)

output_fps = 30


def resize_image(image, dimensions):
    return tf.image.resize(image, dimensions)

def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = image[tf.newaxis, :]
    return image

def tensor_to_image(tensor):
    tensor = tensor * 255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return tensor

def style_transfer(content_image, style_image):
    # Nota: il tuo modello personalizzato non sembra avere un metodo call che restituisce un'immagine stilizzata direttamente.
    # Potresti dover adattare questa funzione per lavorare con il tuo modello personalizzato.
    stylized_image = model([tf.constant(content_image), tf.constant(style_image)])  # Modifica questa linea se necessario
    return stylized_image  # Assicurati che questa funzione restituisca l'immagine stilizzata



# Funzione per processare un frame del video
def process_frame(frame):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_tensor = tf.convert_to_tensor(frame_rgb)
    frame_tensor = tf.image.convert_image_dtype(frame_tensor, tf.float32)
    frame_tensor = frame_tensor[tf.newaxis, :]
    return frame_tensor


writer = imageio.get_writer('styled_video.mp4', fps=output_fps)

frame_count = 0
start_time = time.time()

style_image_resized = resize_image(style_image_tensor, (300, 300))


while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Processa il frame
    content_image = process_frame(frame)

    content_image = resize_image(content_image, (300, 300))

    # Esegui il trasferimento di stile
    output = style_transfer(content_image, style_image_resized)

    # Converte l'output in una immagine
    output_frame_np = tensor_to_image(output)

    # Scrivi il frame sul disco usando imageio
    writer.append_data(output_frame_np)

    # Aggiorna il contatore di frame
    frame_count += 1

    # Calcola e stampa i FPS ogni secondo
    elapsed_time = time.time() - start_time
    if elapsed_time > 1:
        fps = frame_count / elapsed_time
        print(f'FPS: {fps:.2f}')
        # Resetta il contatore di frame e l'ora di inizio per il prossimo intervallo
        frame_count = 0
        start_time = time.time()

# Rilascia le risorse
cap.release()
writer.close()

# Scarica il video elaborato
files.download('styled_video.mp4')


FPS: 4.54
FPS: 12.35
FPS: 16.74
FPS: 16.78
FPS: 15.93
FPS: 15.88
FPS: 16.47
FPS: 15.81
FPS: 16.34
FPS: 16.38
FPS: 16.21
FPS: 15.91
FPS: 11.47
FPS: 10.59
FPS: 10.57
FPS: 14.66
FPS: 16.61
FPS: 15.91
FPS: 15.72
FPS: 16.01
FPS: 16.06


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:


# Carica il video
uploaded = files.upload()
video_name = list(uploaded.keys())[0]

# Salva il video in una variabile
video_data = uploaded[video_name]

# Scrivi il video su disco temporaneamente per poterlo utilizzare con cv2
with open("video.mp4", "wb") as f:
  f.write(video_data)

Saving video.mp4 to video.mp4


In [ ]:
cap = cv2.VideoCapture('video.mp4')


In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_hub as hub
import cv2
import numpy as np
import imageio
import time

output_fps = 30


def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = image[tf.newaxis, :]
    return image

def tensor_to_image(tensor):
    tensor = tensor * 255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return tensor

def style_transfer(content_image, style_image):
    stylized_image = model(tf.constant(content_image), tf.constant(style_image))[0]
    return stylized_image

#def style_transfer(content_image, style_image):
#    stylized_image = model(content_image, style_image)
#    return stylized_image


# Funzione per processare un frame del video
def process_frame(frame):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_tensor = tf.convert_to_tensor(frame_rgb)
    frame_tensor = tf.image.convert_image_dtype(frame_tensor, tf.float32)
    frame_tensor = frame_tensor[tf.newaxis, :]
    return frame_tensor


writer = imageio.get_writer('styled_video.mp4', fps=output_fps)

frame_count = 0
start_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Processa il frame
    content_image = process_frame(frame)

    # Esegui il trasferimento di stile
    output = style_transfer(content_image, style_image_tensor)

    # Converte l'output in una immagine
    output_frame_np = tensor_to_image(output)

    # Scrivi il frame sul disco usando imageio
    writer.append_data(output_frame_np)

    # Aggiorna il contatore di frame
    frame_count += 1

    # Calcola e stampa i FPS ogni secondo
    elapsed_time = time.time() - start_time
    if elapsed_time > 1:
        fps = frame_count / elapsed_time
        print(f'FPS: {fps:.2f}')
        # Resetta il contatore di frame e l'ora di inizio per il prossimo intervallo
        frame_count = 0
        start_time = time.time()

# Rilascia le risorse
cap.release()
writer.close()

# Scarica il video elaborato
files.download('styled_video.mp4')

FPS: 0.41
FPS: 3.66
FPS: 3.74
FPS: 3.72
FPS: 3.68
FPS: 3.68
FPS: 3.69
FPS: 3.68
FPS: 3.61
FPS: 3.59
FPS: 3.58
FPS: 3.64
FPS: 3.48
FPS: 3.62
FPS: 3.60
FPS: 3.52
FPS: 3.55
FPS: 3.61
FPS: 3.48
FPS: 3.56
FPS: 3.19
FPS: 2.63
FPS: 2.81
FPS: 3.47
FPS: 3.46
FPS: 3.56
FPS: 3.55
FPS: 3.57
FPS: 3.58
FPS: 3.57
FPS: 3.49
FPS: 3.41
FPS: 3.28
FPS: 2.85
FPS: 1.82
FPS: 3.31
FPS: 3.52
FPS: 3.59
FPS: 3.53
FPS: 3.57
FPS: 3.64
FPS: 3.61
FPS: 3.60
FPS: 3.54
FPS: 3.27
FPS: 2.83
FPS: 3.28
FPS: 3.64
FPS: 3.65
FPS: 3.66
FPS: 3.66
FPS: 3.64
FPS: 3.60
FPS: 3.69
FPS: 3.65
FPS: 3.65
FPS: 3.02
FPS: 3.25
FPS: 3.48
FPS: 3.62
FPS: 3.60
FPS: 3.64
FPS: 3.67
FPS: 3.70
FPS: 3.67
FPS: 3.66
FPS: 3.63
FPS: 3.61
FPS: 2.77
FPS: 2.82
FPS: 3.34


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os

num_cores = os.cpu_count()
print(f"Numero di core della CPU: {num_cores}")


Numero di core della CPU: 2


In [ ]:
print(type(output))
print(output)



# Converti l'immagine PIL in un array NumPy
output_array = np.array(output)

# Converti l'array da RGB a BGR (formato OpenCV)
output_frame = cv2.cvtColor(output_array, cv2.COLOR_RGB2BGR)

# Salva il frame elaborato
out.write(output_frame)

NameError: ignored

In [ ]:
from google.colab import files

# Salva il frame come immagine
cv2.imwrite('output_frame.jpg', output_frame)

# Scarica il file
files.download('output_frame.jpg')


NameError: ignored